In [ ]:
import pandas as pd
from google.colab import drive
!pip install pyreadr
!pip install category_encoders
from category_encoders import TargetEncoder
import pyreadr
drive.mount('/content/gdrive')
pd.options.display.max_columns = 50
import numpy as np
from openpyxl import Workbook, load_workbook
from datetime import datetime
from sklearn.preprocessing import PowerTransformer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.9/440.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 697.7 kB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
import pandas as pd
data3 = pd.read_csv('/content/gdrive/My Drive/Teachers_Model_B/dpedB_data_90.csv')



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data3.info()
data3 = data3.dropna()

In [ ]:

# Training and testing data
start_date_train = '2022-07-01'
end_date_train = '2022-11-30'

training_data = data3[(data3['REPORT_DATE'] >= start_date_train) & (data3['REPORT_DATE'] <= end_date_train)]
# training_data.dropna(inplace=True)
# Validation set data
start_date_val= '2022-12-01'
end_date_val = '2023-01-30'
Val_data = data3[(data3['REPORT_DATE'] >= start_date_val) & (data3['REPORT_DATE'] <= end_date_val)]

# Target data
target_train = training_data.DPD90
target_Val = Val_data.DPD90


In [ ]:
# Dropping target and date
training_data.drop(columns = ['REPORT_DATE', 'DPD90'], inplace = True)
Val_data.drop(columns = ['REPORT_DATE', 'DPD90'], inplace = True)

imputer = SimpleImputer(strategy='mean')
Val_data .replace([np.inf, -np.inf], np.nan, inplace=True)

Val_data  = imputer.fit_transform(Val_data )


<ipython-input-31-05212b49e505>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.drop(columns = ['REPORT_DATE', 'DPD90'], inplace = True)
<ipython-input-31-05212b49e505>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Val_data.drop(columns = ['REPORT_DATE', 'DPD90'], inplace = True)
<ipython-input-31-05212b49e505>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Val_data .replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:
# Model design Xgboost

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, log_loss, mean_squared_error, mean_absolute_error
import xgboost as xgb

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
pd.set_option('display.float_format', lambda x: '%.6f' % x)


from imblearn.under_sampling import RandomUnderSampler
import numpy as np


from sklearn.linear_model import LinearRegression

from sklearn.impute import SimpleImputer

# Unsampled data for prediction

training_data.replace([np.inf, -np.inf], np.nan, inplace=True)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(training_data)


X_train1, X_test1, y_train1, y_test1 = train_test_split(X_imputed, target_train, test_size=0.20, random_state=42)



model = LinearRegression()

# Training the model
model.fit(X_train1, y_train1)





<ipython-input-32-c8058ad71c88>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.replace([np.inf, -np.inf], np.nan, inplace=True)


LinearRegression()

In [ ]:

def score(expected,predicted,cutoff):
    target_0=len([x for x in expected if x==0])
    target_1=len([x for x in expected if x==1])
    bad_rate=target_1/(target_1+target_0)

    fpr, tpr, threshold_train = roc_curve(expected,predicted)
    roc_auc = auc(fpr, tpr)
    gini = (roc_auc*2-1)
    ks = max(tpr - fpr)
    predicted_bool=[1 if x>=cutoff else 0 for x in predicted]
    Accuracy=accuracy_score(expected,predicted_bool)
    Precision=precision_score(expected,predicted_bool)
    Sensitivity=recall_score(expected,predicted_bool)
    Specificity=recall_score(expected,predicted_bool, pos_label=0)
    F1=f1_score(expected,predicted_bool)
    Balanced_accuracy=balanced_accuracy_score(expected,predicted_bool)
    cm=confusion_matrix(expected,predicted_bool)
    TP=cm[1][1]
    FP=cm[0][1]
    TN=cm[0][0]
    FN=cm[1][0]
    metrics=[target_0,target_1,bad_rate,roc_auc,gini,ks, cutoff,Accuracy,Balanced_accuracy,Sensitivity,Specificity,Precision,F1,TP,FP,FN,TN]
    return metrics


# Predict on the test set
y_pred_test = model.predict(X_test1)
# y_pred_prob_test = model.predict_proba(X_test1)[:, 1]
# Predit on the VaI data
y_pred_v = model.predict(Val_data)
# y_pred_prob_v = model.predict_proba(Val_data)[:, 1]
# Predict on train
y_pred_train = model.predict(X_train1)
# y_pred_prob_train = model.predict_proba(X_train1)[:, 1]

cutoff=0.04
index=['Majority_obs','Minority_obs','Bad_rate','AUC','Gini','KS', 'Cut_off','Accuracy','Balanced_Accuracy','Sensitivty','Specificity','Precision','F1',
         'TP','FP','FN','TN']
score_test=score(y_test1,y_pred_test,cutoff)
score_train=score(y_train1,y_pred_train,cutoff)

score_oot=score(target_Val, y_pred_v,cutoff)

scores=np.array([score_train,score_test,score_oot])
pd.DataFrame(np.transpose(scores), columns = ['Train' , 'Test' ,'OOT'], index=index)

,Train,Test,OOT
Majority_obs,805347.000000,201202.000000,258401.000000
Minority_obs,101368.000000,25477.000000,32395.000000
Bad_rate,0.111797,0.112392,0.111401
AUC,0.929742,0.929837,0.901001
Gini,0.859484,0.859674,0.802002
KS,0.732885,0.730790,0.677024
Cut_off,0.040000,0.040000,0.040000
Accuracy,0.710829,0.711755,0.751245
Balanced_Accuracy,0.802024,0.802405,0.800402
Sensitivty,0.919481,0.919339,0.863652
